In [1]:
import numpy as np
import pandas as pd
from pyecharts import TreeMap

In [2]:
data = pd.read_csv( 'US Stock Info.csv' )
data.head()

,code,market_cap,return,sector
0,AAPL,8.860000e+11,0.012412,TECHNOLOGY
1,ABT,1.430000e+11,0.006442,HEALTHCARE
2,GOLD,2.360000e+10,-0.006642,BASIC_MATERIALS
3,ADSK,3.630000e+10,-0.001089,TECHNOLOGY
4,ADBE,1.370000e+11,0.007730,TECHNOLOGY


In [3]:
def print_groups( group_obj ):
    for name, group in group_obj:
        print( name )
        print( group.head() )

In [4]:
grouped = data.groupby(['sector'])
print_groups(grouped)

BASIC_MATERIALS
     code    market_cap    return           sector
2    GOLD  2.360000e+10 -0.006642  BASIC_MATERIALS
16    APD  4.870000e+10 -0.007310  BASIC_MATERIALS
55    ECL  5.760000e+10  0.004327  BASIC_MATERIALS
115   NEM  2.890000e+10 -0.011208  BASIC_MATERIALS
133   PPG  2.700000e+10  0.003070  BASIC_MATERIALS
COMMUNICATION_SERVICES
      code    market_cap    return                  sector
25     BCE  4.210000e+10 -0.001067  COMMUNICATION_SERVICES
42   CMCSA  1.870000e+11 -0.004099  COMMUNICATION_SERVICES
141      T  2.330000e+11 -0.017549  COMMUNICATION_SERVICES
195   DISH  1.750000e+10  0.005925  COMMUNICATION_SERVICES
201    RCI  2.740000e+10  0.000000  COMMUNICATION_SERVICES
CONSUMER_CYCLICAL
   code    market_cap    return             sector
20  AZO  2.730000e+10  0.011951  CONSUMER_CYCLICAL
24  BBY  1.740000e+10  0.022599  CONSUMER_CYCLICAL
29  BLL  2.220000e+10  0.006373  CONSUMER_CYCLICAL
52  DIS  2.470000e+11 -0.006956  CONSUMER_CYCLICAL
62    F  3.920000e+10  0.005

In [5]:
group_value = grouped.apply( lambda df: np.sum(df['market_cap']) )
group_value

sector
BASIC_MATERIALS           3.367000e+11
COMMUNICATION_SERVICES    1.067600e+12
CONSUMER_CYCLICAL         2.946600e+12
CONSUMER_DEFENSIVE        2.150900e+12
ENERGY                    1.270500e+12
FINANCIAL_SERVICES        4.461700e+12
HEALTHCARE                3.076000e+12
INDUSTRIALS               2.365500e+12
REAL_ESTATE               5.053000e+11
TECHNOLOGY                5.941700e+12
UTILITIES                 6.931000e+11
dtype: float64

In [6]:
data_for_treemap = []

for i, grp in enumerate(grouped):
    
    total_mktcap, sector = group_value[i], group_value.index[i]
    code, r, mktcap = grp[1]['code'], grp[1]['return'], grp[1]['market_cap']

    children = []
    
    for j in range(len(code)):
        
        j_data = { 'value' : mktcap.iloc[j], 
                   'name': [sector, code.iloc[j], r.iloc[j]] }
        
        children.append( j_data )
    
    i_data = { 'value' : total_mktcap,
               'name': sector,
               'children': children }
    
    data_for_treemap.append( i_data )

data_for_treemap

[{'value': 336700000000.0,
  'name': 'BASIC_MATERIALS',
  'children': [{'value': 23600000000.0,
    'name': ['BASIC_MATERIALS', 'GOLD', -0.006642]},
   {'value': 48700000000.0, 'name': ['BASIC_MATERIALS', 'APD', -0.00731]},
   {'value': 57600000000.0, 'name': ['BASIC_MATERIALS', 'ECL', 0.004327]},
   {'value': 28900000000.0, 'name': ['BASIC_MATERIALS', 'NEM', -0.011208]},
   {'value': 27000000000.0, 'name': ['BASIC_MATERIALS', 'PPG', 0.00307]},
   {'value': 43500000000.0, 'name': ['BASIC_MATERIALS', 'SHW', -0.004979]},
   {'value': 17900000000.0, 'name': ['BASIC_MATERIALS', 'VMC', 0.009007]},
   {'value': 28100000000.0, 'name': ['BASIC_MATERIALS', 'SCCO', 0.009154]},
   {'value': 31000000000.0, 'name': ['BASIC_MATERIALS', 'LYB', 0.056313]},
   {'value': 30400000000.0, 'name': ['BASIC_MATERIALS', 'NTR', 0.004647]}]},
 {'value': 1067600000000.0,
  'name': 'COMMUNICATION_SERVICES',
  'children': [{'value': 42100000000.0,
    'name': ['COMMUNICATION_SERVICES', 'BCE', -0.001067]},
   {'valu

In [7]:
def label_formatter( params ):
    sign = '+' if params.name[2] > 0 else ''
    return params.name[1] +'\n' \
           + sign + window.parseFloat(params.name[2]*100).toFixed(2) + r'%'

In [8]:
def tooltip_formatter( params ):
    up_flat_down = 'up' if params.name[2] > 0 else 'down' if params.name[2] > 0 else 'flat'
    return params.name[1] \
           + '<br>' + 'Sector: ' + params.name[0] \
           + '<br>' + 'Market Cap: ' + window.parseFloat(params.value/1000000000) + ' bio' \
           + '<br>' + 'Direction: ' + up_flat_down

In [9]:
treemap = TreeMap( width=1600, height=700 )

treemap.add( 'Stock Tree Map', data_for_treemap, is_random=False, 
              is_legend_show=False, line_opacity=0.5, is_label_show=True, 
              label_text_size=12, label_pos='inside', legend_orient='vertical',
              tooltip_formatter=tooltip_formatter,
              label_formatter=label_formatter )

#treemap.render(path=u'TreeMap.html')